# Extract Documentation Text

#### Imports

In [233]:
from pathlib import Path
from functools import partial
from itertools import chain
from typing import  List, Union
from pickle import load, dump
import re

import pandas as pd
import xlwings as xw

from true_iterable import true_iterable
from sectionary.text_reader import FixedWidthParser, define_fixed_width_parser
from sectionary.text_reader import trim_items, drop_blanks, to_dataframe
from sectionary.sections import Section, SectionBreak, ProcessingMethods, Rule
from sectionary.sections import Trigger
from sectionary.buffered_iterator import BufferedIterator, BufferedIteratorEOF

## Load the Varian DataTypes

In [234]:
def get_data_types():
    PKL_DIR = Path.cwd() / '..' / 'Raw Tables'
    data_type_pkl_file = PKL_DIR / 'Data Type.pkl'

    file = open(str(data_type_pkl_file), 'rb')
    data_type_lookup = load(file)
    file.close()
    return data_type_lookup

### Convert the DataType definition table into a dictionary that links type name with category.

In [235]:
def get_data_type_dict():   
    data_types = get_data_types().reset_index()
    # Get the two datatype groups
    data_types.columns = ['VDT', 'MSQL_DT']
    # Convert to a dictionary look up of category
    data_types = data_types.T.stack().reset_index(level=0)
    data_types.columns = ['Category', 'DataType']
    data_types.set_index('DataType', inplace=True)
    return data_types.to_dict()['Category']

# General Text Cleaning

## Remove Line Break text

In [236]:
def drop_lines(lines: List[str])->str:
    brk_text = [
        'System Database Reference Guide',
        'VDT Name to Microsoft SQL Datatype',
        'Abbreviations for Table and Column Names',
        'variansystem Database Overview',
        'Varian System Database Reference Guide',
        '(poster only)',
        '(continued)',
        'ARIA modules installed.',
        'This page is intentionally blank'
        ]
    for line in lines:
        if any(drop_text in line 
               for drop_text in brk_text):
            continue
        else:
            yield line

### Remove lines based on regular expressions
1. Lines that end in a number (page number).
2. Lines containing spaces and just the text _Name_ 
(_View Column Name_ wrapped to the next line.)

In [237]:
def drop_lines_re(lines: List[str])->str:
    # convert single string to one element list of strings
    re_list = [
        ' +[0-9]+$',  # Lines ending in a number
        #'^[ ]+Name$'  # Lines containing spaces and just the text Name
        ]
    for line in lines:
        # Replace invalid UTF code with black circle
        # line = line.replace('\uf06e', u'\u25cf')
        if any(re.search(test_re, line) is not None
               for test_re in re_list):
            continue
        else:
            yield line

## Clean Row Text
1. Remove the spaces around KeyType annotation.
2. Fix the UserDefAttrib column name(s)
    ```
    UserDefAttrib01 
           to       
    UserDefAttrib16 
    ```
3. Fix the single instance of comma separated KeyTypes
4. Add _ to a single instance of Data Type
4. Correct case of a single instance of Data Type
5. Remove spacing before _ in a single instance of Data Type
5. Replace - with _ in a single instance of Data Type
4. Add extra row when datatype is _image_, because it only uses one row for a column definition.

In [238]:
def clean_row_text(table_data_list):
    for row in table_data_list:
    # Drop spaces around keytypes
        if ' (' in row:
            row = row.replace(' (', '(') 
            row = ' ' + row
        # re-label the UserDefAttrib01 column
        row = row.replace('UserDefAttrib01     ', 'UserDefAttrib[01-16]')
        row = row.replace('       to           ', '                    ')
        row = row.replace('UserDefAttrib16     ', '                    ')
        # replace comma seperated key type codes with the standard brackets
        row = row.replace('(FK, IE1)', '(FK)(IE1)')
        # Add _ to Data Type
        row = row.replace('VDT_MODALITY NOT ', 'VDT_MODALITY_NOT_')
        # remove spacing before _ in Data Type
        row = row.replace('VDT_FLAG_TRUE _', 'VDT_FLAG_TRUE_')        
        # Correct DataType Name case
        row = row.replace('VDT_Name', 'VDT_NAME')        
        # Replace - with _
        row = row.replace('VDT_FLAG_FALSE-DEFAULT', 'VDT_FLAG_FALSE_DEFAULT')
        # Add an extra line when type is image (only one line for a column def)
        if row.endswith('image'):
            yield row
            yield ''
            continue
        yield row

# Define the Chapter Sections

## Chapter data extraction process:
1. Get all text between Chapter 4 and 14.
> `raw_text` -> `remaining_lines`
3. Extract all text starting just after the next chapter title line.
> `remaining_lines` -> `next_chapter` -> `remaining_lines`.
3. Get the name of the zone from the chapter title.
> (as found by `next_chapter`)
1. Extract text for the current chapter starting at the beginning of the text selected in 2. and ending just before the next next chapter title line.(`remaining_lines` -> `this_chapter` -> `chapter_lines`)


## All chapters
> Selects the section of raw text containing the chapters of interest

In [239]:
all_chapters = Section(
    start_section=SectionBreak('CHAPTER 4', break_offset='Before'), 
    end_section=SectionBreak('Chapter 14', break_offset='Before')
    )

## Define **Next** chapter section
> Collect all chapter contents starting right after the chapter title line.
- Start just after the chapter title.
- Continue to end of text
- Remove page headers & footers

### Regular expression for chapter titles
- Looks for the word `Chapter` followed by numbers 4 thru 99.
- Capture the chapter title which follows the chapter number.

In [240]:
chapter_loc = re.compile(
    '^ +'          # Spaces
    'Chapter '     # Chapter Text
    '(?:'          # Begin non-capture group
    '[4-9]'        # Chapters 4 thru 9
    '|'            # or
    '[1-9][0-9]'   # Chapters 10 and up
    ')'            # End non-capture group
    ' +'           # Blank spaces
    '(.+)$'        # The rest of the text on the line
    )

### Chapter processing methods:
- Remove page headers & footers
- Apply misc. text cleaning

In [241]:
# Remove page headers & footers
next_ch_proc = ProcessingMethods([drop_lines, drop_lines_re, clean_row_text])

next_chapter = Section(
    start_section=SectionBreak(chapter_loc, break_offset='After'), 
    end_section=None,
    processor=next_ch_proc
    )

### Extract the zone label from the chapter title
- The chapter title is _captured_ by the chapter_loc regular expression and 
can be accessed with `next_chapter.context['Event'].groups()[0]`
- The text `Zone Tables` or just `Zone` is removed.
- Starting and ending spaces are removed
- The end result is assumed to be the name of the zone.

In [242]:
def get_zone_label(ch_section):
    chapter_title = ch_section.context['Event'].groups()[0]
    zone_label = chapter_title.replace('Zone Tables','').strip()
    zone_label = zone_label.replace('Zone','').strip()
    return zone_label

## Define **This** chapter section
> Returns the text for the current chapter
- Start with beginning of text.
- End just before next chapter title.
- No Text processing is done

### _this_ chapter section definition

In [243]:
this_chapter = Section(
    start_section=None, 
    end_section=SectionBreak(chapter_loc, break_offset='before')
    )

## Function to read a chapter

In [244]:
def read_chapter(remaining_lines):
    remaining_lines = next_chapter.read(remaining_lines)
    if remaining_lines:
        zone = get_zone_label(next_chapter)
        chapter_lines = this_chapter.read(remaining_lines)
    else:
        chapter_lines = []
        zone = ''
    return remaining_lines, chapter_lines, zone

# Define the Table Sections

### Make A regular expression for Table headers
- Starts with 10-20 Spaces.
- Capture the table name:
    - Begins with a capital letter or "vv_".
    - Contains no spaces or period for the rest of the line.

In [245]:
table_loc = re.compile(''.join([
    '^',           # Start of line
    ' {10,20}',    # between 10 and 20 Spaces
    '('            # Start of capture group
    '([A-Z]|vv_)'  # Begins with a capital letter or "vv_"
    '[^ .]+'       # Text with no spaces or period (for rest of line)
    ')'            # End of capture group
    '$'            # End of the line
    ]))

## Table Section Definition
> Returns text starting with the next table
- Start after the next Table name (as defined by `table_loc`)
- Continue to the end (of the chapter)
- No text processing is done.

In [246]:
table_section = Section(
    start_section=SectionBreak(table_loc, break_offset='After'), 
    end_section=None,
    )

### Extract the Table Name from the table section
- The table name is _captured_ by the table_loc regular expression and can be 
accessed with `table_section.context['Event'].groups()[0]`
- Starting and ending spaces are removed.

In [247]:
def get_table_name(tbl_section):
    if tbl_section.context['Event']:
        table_name = tbl_section.context['Event'].groups()[0]
        table_name = table_name.strip()
    else:
        table_name = ''    
    return table_name

### Extract the table description text (Description section)
> The table description is a single paragraph immediately after the 
table name.<p>
Obtained by:<p> `table_description = desc_section.read(chapter_lines)`

- Start at the beginning of the table text
- Continue until reaching a line that starts with more than 44 spaces.
- Text processing:
    1. Remove spaces at the beginning and end of each row.
    2. Combine rows to form a single string. 

In [248]:
def trim_row(row):
    return row.strip()
desc_proc = ProcessingMethods([trim_row])  # Remove leading and trailing spaces

def merged_desc(rows):
    return ' '.join(rows)

desc_section = Section(
    start_section=None, 
    end_section=SectionBreak(' '*44, break_offset='Before'),
    processor=desc_proc, aggregate=merged_desc
    )

### Function to read table information

In [249]:
def get_table_info(chapter_lines, zone):
    chapter_lines = table_section.read(chapter_lines)
    if chapter_lines:
        table_name = get_table_name(table_section)
        table_description = desc_section.read(chapter_lines)
        table_info = {
            'Zone': zone,
            'TableName': table_name,
            'TableDescription': table_description
            }
    else:
        table_info = {}
    return chapter_lines, table_info

# Define the Table's Column Section
### Remove table header text
> Table header contains either:

> - Column Name
> - View Column

> If header is found return a blank string, otherwise return the original row.

### Column Section
> Returns text containing the column descriptions
- Start just after the Table Header
- Continue to just before the next table
- Table headers and blank lines are removed.

In [250]:
drop_table_header = Rule(['Column Name', 'View Column'],
                         pass_method='Blank',
                         fail_method='Original')

col_proc = ProcessingMethods([
     drop_table_header,  # Remove table header text (repeats at beginning of page)
     drop_blanks         # Remove lines where all columns are empty.
    ])

tbl_columns = Section(
    start_section=SectionBreak(['Column Name', 'View Column'], 
                               break_offset='After'), 
    end_section=SectionBreak(table_loc, break_offset='Before'),
    processor=col_proc
    )

## Identify Data Types
> Find DataType names at the end of description text

In [251]:
def type_search(row, data_types):    
    desc_txt = row.get('DescTxt','')
    if isinstance(desc_txt, str):
        for data_type, category in data_types.items():
            # Remove 'image' as a searchable data type 
            # (it is a word in its own right)
            if data_type == 'image':
                continue
            # Add space in front so that we don't find partial words
            test_text = ' ' + data_type
            if str(desc_txt).endswith(test_text):
                idx = desc_txt.rfind(test_text)
                row[category] = desc_txt[idx:]
                row['DescTxt'] = desc_txt[:idx]
    return row

# Read All Tables in All Chapters
### Chapter data extraction process:
1. Get all text between Chapter 4 and 14.
2. For each chapter:
    1. Get the name of the zone from the chapter title.
    2. Extract all text starting just after the next chapter title line.
    3. Extract text for the current chapter starting at the beginning of the text selected in 2. and ending just before the next next chapter title line.
    4. For each table in the chapter:
        1. Return text starting with the next table
        2. Get the table Name and description
        3. Extract the column description text
        4. Add Zone and Table Name to the beginning of each row
3. Build the Table description DataFrame
4. Create a list of column descriptions for each column in all chapters

In [252]:
def read_chapters(raw_lines):
    table_desc_list = list()
    table_data_list = list()
    table_ref_tmpl = '{zone:25}{table_name:30}'
    remaining_lines = all_chapters.read(raw_lines)
    while remaining_lines:
        remaining_lines, chapter_lines, zone =  read_chapter(remaining_lines)
        while chapter_lines:
            chapter_lines, table_info = get_table_info(chapter_lines, zone)
            if chapter_lines:
                table_data = tbl_columns.read(chapter_lines)                
                table_ref = table_ref_tmpl.format(
                    zone=table_info['Zone'],
                    table_name=table_info['TableName']
                    )
                combined_table_data = [table_ref + row for row in table_data]
                table_desc_list.append(table_info)
                table_data_list.extend(combined_table_data)
    table_descriptions = pd.DataFrame(table_desc_list)
    return table_descriptions, table_data_list

# Extract a Column Definition

## Column locator regular expression
- allow for 2 to 20 Spaces at the beginning of the line.
- ColumnName is the sequence of all subsequent non-space characters
- ColumnName is followed by at least one space
- DescTxt contains the extracted beginning of the description text.
- DescTxt begins with at least one non-space character, followed by all 
remaining characters in the extracted line

In [253]:
col_loc = re.compile(
    # Initial spaces
    '^'                # Start of line
    # StartSpace  
    '(?P<StartSpace>'  # beginning of StartSpace group
    ' {2,20}'          # 2 to 20 Spaces
    ')'                # End of StartSpace group  
      
    # ColumnName  
    '(?P<ColumnName>'  # beginning of ColumnName group
    '[^ ]+'            # all non-space characters
    ')?'               # End of optional ColumnName group
  
    # GapSpace  (Space between Values)
    '(?P<GapSpace>'  # beginning of GapSpace group
    ' +'               # At least one space
    ')'                # End of GapSpace group
                
    # Some ColumnDescription text
    '(?P<DescTxt>'     # beginning of DescTxt group
    '[^ ]+'            # at least one non-space characters
    '.*'               # all remaining characters
    ')?'               # End of optional DescTxt group
    
    # End of Line  
    '$'                # End of line
    )

## DataType regular expression

1. Column Description (DescTxt)
- All characters from the start of the line until next section is found (non-greedy)
2. An arbitrary number of spaces
3. DataType group
- DataType can be either VDT or MSQL_DT 
- VDT  
    - Group must be preceded by at least 2 spaces
    - Contains any number of capital letters, numbers, symbols: _ ) (
- MSQL_DT  
    - Group must be preceded by at least 2 spaces
    - Starts with a letter followed by a lowercase letter
    - All remaining text in the line

In [254]:
type_loc = re.compile(
    # Initial spaces
    '^'             # Start of line
    # DescTxt  
    '(?P<DescTxt>'  # beginning of ColumnDescription group
    '.+?'           # all characters until next section is found (non-greedy)
    ')?'            # End of optional ColumnDescription group
      
    # Space between Values  
    ' *'            # Arbitrary number of Spaces
      
    # DataType can be either VDT or MSQL_DT  
    '('             # beginning of DataType group
    # VDT  
    '(?P<VDT>'      # beginning of VDT group
    '(?<=  )'       # Group must be preceded by at least 2 spaces
    '[A-Z_0-9)()]+' # capitals numbers and "-)("
    ')'             # End of VDT group
    
    '|'             # OR    
    # MSQL_DT  
    '(?P<MSQL_DT>'  # beginning of MSQL_DT group
    '(?<=  )'       # Group must be preceded by at least 2 spaces
    '[A-Za-z]'      # Starts with a letter
    '[a-z]'         # followed by a lowercase letter
    '.+'            # remaining text in line
    ')'             # End of MSQL_DT group

    ')?'            # End of optional DataType group
      
    # End of Line  
    '$'             # End of line
    )

## Extract Values from Each Row
1. Use positions to get _Zone_ and _TableName_
2. Use `col_loc` to get _ColumnName_ and part of _Description_
3. Use `typ_match` to get _VDT_, _MSQL_DT_ and part of _Description_
4. Combine _Description_ parts
6. Calculate _DescriptionOffset_

In [255]:
def get_row_data(table_data_iter):
    def get_table_dict(info_txt):
        zone = info_txt[:25].strip()
        table_name = info_txt[25:].strip()
        table_dict = {
            'Zone': zone,
            'TableName': table_name
            }
        return table_dict
    
    def get_col_match(col_row):
        col_match = col_loc.match(col_row)
        if col_match:
            match_result = col_match.groupdict(default='')     
        else:
            match_result = {}
        return match_result
    
    def get_type_match(typ_row, data_types):
        typ_match = type_loc.match(typ_row)
        if typ_match:
            match_result = typ_match.groupdict(default='')
            final_match_result = type_search(match_result, data_types)        
        else:
            final_match_result = {}
        return final_match_result

    try:
        row = next(table_data_iter)
    except BufferedIteratorEOF:
        return {}
    if not row:
        return {}
    row_dict = get_table_dict(row[0])
    if len(row) > 1:
        col_data = get_col_match(row[1])
        row_dict['ColumnName'] = col_data['ColumnName']
        row_dict['Description'] = [col_data['DescTxt']]
        row_dict['DescriptionOffset'] = [len(col_data['DescTxt'])]           
        if len(row) > 2:
            type_data = get_type_match(row[2], data_types)
            row_dict['VDT'] = type_data['VDT']
            row_dict['MSQL_DT'] = type_data['MSQL_DT']
            row_dict['Description'] = [''.join([col_data['DescTxt'],
                                                type_data['DescTxt']])]
    return row_dict

## Check Type Word Wrap
1. Merge current type text with next row type text
2. If resulting merge is not blank:
    - If True:  Check if resulting merge is a valid type:
       - If True:  Return resulting merge an new type text
       - Otherwise:  flag _New Column_
    - Otherwise:   original type text (blank) is kept

In [256]:
def check_data_types(row_dict, next_row_dict, data_types):
    def get_type(row_dict, type_category):
        tp = row_dict.get(type_category, '')
        tp = tp.strip()
        return tp

    def merge_type(row_dict, next_row_dict, type_category, data_types):
        # Treat type_data as a word wrap of one data type name
        type_data = [get_type(row_dict, type_category),
                     get_type(next_row_dict, type_category)]           
        type_text = ''.join(type_data)
        
        # If type_text is blank return type_text
        if not type_text:            
            return type_text
        
        # If type_text is a valid data type in its category return type_text
        for data_type, category in data_types.items():            
            if ((type_text.lower() in data_type.lower()) & 
                (type_category.lower() in category.lower())):
                return type_text
            
        # If type_text is not a valid data type check whether it may be the end 
        # of the column name by testing for ColumnName text
        if not next_row_dict['ColumnName']:
                # If ColumnName is blank, treat type_data as two data types 
                # and join with a comma and space
                type_text = ', '.join(type_data)
                return type_text
        
        # If ColumnName is not blank, assume that the end of the column name 
        # has been reached and return None
        return None
    
    new_column = False
    vdt = merge_type(row_dict, next_row_dict, 'VDT', data_types)
    if vdt is not None:
        row_dict['VDT'] = vdt
    else:
        new_column = True
    msql = merge_type(row_dict, next_row_dict, 'MSQL_DT', data_types)
    if msql is not None:
        row_dict['MSQL_DT'] = msql   
    else:
        new_column = True
    return row_dict, new_column

## Get full column definition
1. Check for end of table_data_iter 
2. Check for end of table
3. Check for end of column<p>
    _Start of Column_<p>
    1. Start with<p>
        `end_of_name = False`
    2. Read rows until<p>
    	`not next_row_dict['ColumnName']`
    3. Set<p>
    	`end_of_name = True`
    4. Continue to read rows until
    	- `next_row_dict['ColumnName']`
		or
		- Invalid datatype word wrap
    5. Set<p>
    	`table_data_iter.backup()`<p>

    _End of Column_<p>

4. Merge column rows to form column definition dictionary

In [257]:
def get_column_def(table_data_iter):
    row_dict = get_row_data(table_data_iter)
    this_table = row_dict.get('TableName', '')
    end_of_name = False
    while True:    
        next_row_dict = get_row_data(table_data_iter)
        # Check for end of table_data_iter 
        if not next_row_dict:
            break
        # Check for end of table
        if next_row_dict['TableName'] not in this_table:
            table_data_iter.backup()
            break
        # Check for end of column
        if not next_row_dict['ColumnName']:
            end_of_name = True   # Finished ColumnName word wrap
        else:
            if end_of_name:   # Starting new ColumnName
                table_data_iter.backup()
                break            
        row_dict, new_column = check_data_types(row_dict, next_row_dict, 
                                                data_types)
        # Check for end of column indicated by invalid datatype word wrap
        if new_column:
            table_data_iter.backup()
            break      
        # if made it to here then still part of same column definition        
        row_dict['ColumnName'] = ''.join([row_dict['ColumnName'],
                                        next_row_dict['ColumnName']])
        row_dict['Description'].extend(next_row_dict['Description'])
        row_dict['DescriptionOffset'].extend(next_row_dict['DescriptionOffset'])
    return row_dict

## Extract KeyTypes
1. Use a regular expression to identify Key Types in the column name.
2. If one or more Key Types are found:
    1. Update the column name
    1. Put commas between multiple Key Types
    2. Remove Brackets
    3. Set the KeyType value

### Column KeyType Regular Expression
> KeyType identification (Possibly more than one)

- Each KeyType code begins with "(" and ends with ")"
- Usually a space between multiple KeyType codes
- Possible KeyType codes are:
         
|Symbol|Meaning|
|------|-------|
|(PK)|Primary Key|
|(FK)|Foreign Key|
|(AK#)|Alternate Key|
|(IE#)|Inversion Entry|

**Note:** _#_ indicates a number

In [258]:
key_loc = re.compile(
    '^'                # Start of line
    # ColumnName  
    '(?P<ColumnName>'  # beginning of ColumnName group
    '[^ (]+'           # all non-space and non-'(' characters
    ')?'               # End of optional ColumnName group
    '(?P<KeyType>'               # beginning of optional KeyType group
    '('                          # Beginning of keyType    
    '\('                         # KeyType code begins with a "("
    '(PK|FK|A[KE][0-9]?|IE[0-9]?)'  # Possible KeyType codes
    '\)'                         # Each KeyType code ends with a ")"
    ')+'                         # Possibly more than one KeyType Code
    ')?'                         # End of optional KeyType group 
    # End of Line  
    '$'                          # End of line
    )

In [259]:
def get_key_type(column_def):
    key_match = key_loc.match(column_def['ColumnName'])
    if not key_match:
        return column_def
    name_dict = key_match.groupdict(default='')
    # Remove Brackets and put commas between multiple types
    name_dict['KeyType'] = name_dict['KeyType'].replace(')(',', ')
    name_dict['KeyType'] = name_dict['KeyType'].replace('(','').replace(')','')
    column_def.update(name_dict)
    return column_def

## Apply Column Description formatting
1. Replace original bullet symbols with line breaks and Excel friendly bullets symbols
2. Add line breaks after ":"
3. Convert double line breaks to single line breaks

In [260]:
def merge_description(column_def):
    clean_lines = [line.strip() for line in column_def['Description']]    
    merged_text = ' '.join(clean_lines)
    bullet_break_text = merged_text.replace('\uf06e', chr(10) + u'\u25cf')
    colon_break_text = bullet_break_text.replace(':', ':' + chr(10))
    dbl_break_text = colon_break_text.replace(''.join([chr(10),' ',chr(10)]), 
                                              chr(10))
    if dbl_break_text.endswith('\n'):
        dbl_break_text = dbl_break_text[:-1]    
    column_def['Description'] = dbl_break_text
    return column_def

# Read and Parse the file
1. Get Raw Text
2. Generate the Column Table

## Get Raw Text
1. Read the text file
2. Extract the desired chapters
3. Break each row into 3 regions for analysis
4. prepare to iterate through the rows

In [261]:
# Set the file name
TEXT_DIR = Path.cwd() / '..' / 'Documentation'
#text_file_name = 'Varian System Database Reference Guide V13.6.txt'
text_file_name = 'Varian System Database Reference Guide V13.6 Edited.txt'
text_file_path = TEXT_DIR / text_file_name

# Read the text file
raw_text = text_file_path.read_text(encoding='utf8', errors='ignore')
# Convert to a list of lines
raw_lines = raw_text.splitlines()

# Select the desires chaper region
table_descriptions, table_data_list = read_chapters(raw_lines)
data_types = get_data_type_dict()  # For matching with known datatypes

# Split each line into 3 parts and iterate through them
row_sections = define_fixed_width_parser(locations=[55, 110])
table_data_iter = BufferedIterator(row_sections(table_data_list))

## Generate the Column Table
1. Parse the table and column text
2. Extract Key Types from the column names
3. Merge and format column description lines
4. Convert to a DataFrame table

In [262]:
column_list = list()
while True:
    column_def = get_column_def(table_data_iter)
    if not column_def:
        break
    column_def = get_key_type(column_def)
    column_def = merge_description(column_def)
    column_list.append(column_def)

column_table = pd.DataFrame(column_list)
column_table.drop('DescriptionOffset', axis='columns', inplace=True)

# Save the Table and Column Descriptions
- Save as `.pkl` file 
- Save as an Excel Spreadsheet

### Set the paths for saving data

In [263]:
PKL_DIR = Path.cwd() / '..' / 'Raw Tables'
DATA_DIR = Path.cwd() / '..' / 'Raw Tables'

#### Make pickle File

In [264]:
pickle_file = PKL_DIR / 'Table Descriptions.pkl'
file = open(str(pickle_file), 'wb')
dump(table_descriptions, file)
file.close()

pickle_file = PKL_DIR / 'Column Descriptions.pkl'
file = open(str(pickle_file), 'wb')
dump(column_table, file)
file.close()

#### Make spreadsheet File

In [265]:
# Export to spreadsheet
spreadsheet_file = DATA_DIR / 'Table Descriptions.xlsx'
xw.view(table_descriptions)
workbook = xw.books.active
workbook.sheets.active.name = 'Table Descriptions'
workbook.save(spreadsheet_file)

spreadsheet_file = DATA_DIR / 'Column Descriptions.xlsx'
xw.view(column_table)
workbook = xw.books.active
workbook.sheets.active.name = 'Column Descriptions'
workbook.save(spreadsheet_file)